In [1]:
import csv
import numpy as np
import os
import pandas as pd
import scipy.interpolate
import sklearn.metrics
import sys
sys.path.append("../src")
import localmodule


if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO


from matplotlib import pyplot as plt
%matplotlib inline

# Define constants.
dataset_name = localmodule.get_dataset_name()
models_dir = localmodule.get_models_dir()
units = localmodule.get_units()
n_units = len(units)
n_trials = 10


In [40]:
# Loop over augmentations.
for aug_kind_str in ["none", "all-but-noise", "all"]:
    
    # Loop over models.
    for model_name in [
            "icassp-convnet", "icassp-ntt-convnet", "icassp-add-convnet",
            "pcen-convnet", "pcen-ntt-convnet", "pcen-add-convnet"]:
        
        try:

            if aug_kind_str == "all-but-noise" and model_name == "icassp-convnet":
                continue

            if not aug_kind_str == "none":
                model_name = "_".join([model_name, "aug-" + aug_kind_str])
            model_dir = os.path.join(models_dir, model_name)

            fns, fps, tns, tps, accs = [], [], [], [], []

            for test_unit_id in range(6):
                print(test_unit_id)

                test_unit_str = units[test_unit_id]
                test_unit_dir = os.path.join(model_dir, test_unit_str)

                val_losses = []
                for trial_id in range(n_trials):
                    trial_str = "trial-" + str(trial_id)
                    trial_dir = os.path.join(test_unit_dir, trial_str)
                    history_name = "_".join([
                        dataset_name,
                        model_name,
                        test_unit_str,
                        trial_str,
                        "history.csv"
                    ])
                    history_path = os.path.join(
                        trial_dir, history_name)
                    try:
                        history_df = pd.read_csv(history_path)
                        val_loss = min(-history_df["Validation accuracy (%)"])
                        #val_loss = min(history_df["Validation loss"])
                    except:
                        val_loss = np.inf
                    val_losses.append(val_loss)

                val_losses = np.array(val_losses)
                best_trial_id = np.argmin(val_losses)
                best_trial_str = "trial-" + str(best_trial_id)
                
                validation_units = localmodule.fold_units()[test_unit_id][2]
                predict_folds = [validation_units, [test_unit_id]]
                
                for predict_fold in predict_folds:
                    print(predict_fold)
                    
                    for predict_unit_id in predict_fold:
                        print(predict_unit_id)

                        best_trial_dir = os.path.join(
                            test_unit_dir, best_trial_str)
                        predictions_name = "_".join([
                            dataset_name,
                            model_name,
                            "test-" + test_unit_str,
                            best_trial_str,
                            "predict-" + predict_unit_str,
                            "clip-predictions.csv"
                        ])
                        prediction_path = os.path.join(
                            best_trial_dir, predictions_name)

                        # Load prediction.
                        with open(prediction_path, 'r') as f:
                            reader = csv.reader(f)
                            rows = list(reader)
                        rows = [",".join(row) for row in rows]
                        rows = rows[1:]
                        rows = "\n".join(rows)

                        # Parse rows with correct header.
                        df = pd.read_csv(StringIO(rows),
                            names=[
                                "Dataset",
                                "Test unit",
                                "Prediction unit",
                                "Timestamp",
                                "Center Freq (Hz)",
                                "Augmentation",
                                "Key",
                                "Ground truth",
                                "Predicted probability"])
                        y_pred = np.array(df["Predicted probability"])
                        y_pred = (y_pred > 0.5).astype('int')

                        # Load ground truth.
                        y_true = np.array(df["Ground truth"])

                        # Compute confusion matrix.
                        tn, fp, fn, tp = sklearn.metrics.confusion_matrix(
                            y_true, y_pred).ravel()

                        acc = 100 * (tn+tp) / (tn+tp+fn+fp)

                        print(acc)
                        #tns.append(tn)
                        #fps.append(fp)
                        #fns.append(fn)
                        #tps.append(tp)
                        #accs.append(acc)

                    fn = sum(fns)
                    fp = sum(fps)
                    tn = sum(tns)
                    tp = sum(tps)
                    acc = np.mean(accs)

                    print(model_name, "{:5.2f}%".format(100 * (tn+tp) / (tn+tp+fn+fp)))

        except:
            pass
        
    #print("")

0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07
0
['unit07', 'unit10']
unit07


In [33]:
localmodule.fold_units()[test_unit_id][-1]

['unit07', 'unit10']